In [78]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [95]:
col_names = ["age", "workclass", "fnlwgt", "education", "edu_num", "martial_status", "occupation", "relationship", "race", "gender", 
            "cap_gain", "cap_loss", "hrs_per_week", "country", "salary"]
df = pd.read_csv("adult.csv", names = col_names)
df.head()

,age,workclass,fnlwgt,education,edu_num,martial_status,occupation,relationship,race,gender,cap_gain,cap_loss,hrs_per_week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [96]:
categorical = [i for i in df.columns if df[i].dtype == 'O']
numerical = [i for i in df.columns if df[i].dtype != 'O']

In [97]:
for i in categorical:
    
    print(i+":")
    print(df[i].value_counts())

workclass:
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
education:
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64
martial_status:
 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: martial_status, dtype: int64
occupation:
 Prof-specialty     

In [98]:
df["workclass"].replace(" ?", df["workclass"].mode()[0], inplace = True)
df["occupation"].replace(" ?", df["occupation"].mode()[0], inplace = True)
df["country"].replace(" ?", df["country"].mode()[0], inplace = True)

In [99]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
edu_num           0
martial_status    0
occupation        0
relationship      0
race              0
gender            0
cap_gain          0
cap_loss          0
hrs_per_week      0
country           0
salary            0
dtype: int64

In [100]:
data = [df]
for dataset in data:
    dataset.loc[dataset['country'] != ' United-States', 'country'] = 'Not_US'
    dataset.loc[dataset['country'] == ' United-States', 'country'] = 'US'

In [101]:
x = df.drop("salary", axis=1)
y = df["salary"]

In [102]:
import category_encoders as ce

In [103]:
encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'martial_status', 'occupation', 'relationship', 
                                 'race', 'gender', 'country'])
df = encoder.fit_transform(df)

In [104]:
# for i in numerical:
#     iqr = df[i].quantile(0.75) - df[i].quantile(0.25)
#     low = df[i].quantile(0.25) - (iqr * 1.5)
#     high = df[i].quantile(0.75) + (iqr * 1.5)
#     df[i] = np.where(df[i] > high, high, df[i])
#     df[i] = np.where(df[i] < low, low, df[i])

In [105]:
x = df.drop("salary", axis=1)
y = df["salary"]

In [121]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state = 2)

In [122]:
from sklearn.preprocessing import RobustScaler
scalar = RobustScaler()

xtrain = scalar.fit_transform(xtrain)
xtest = scalar.transform(xtest)

In [123]:
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(xtrain, ytrain)

GaussianNB(priors=None, var_smoothing=1e-09)

In [124]:
y_pred = gnb.predict(xtest)

y_pred

array([' <=50K', ' <=50K', ' >50K', ..., ' >50K', ' <=50K', ' <=50K'],
      dtype='<U6')

In [125]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(ytest, y_pred)))

Model accuracy score: 0.7986
